# TRABALHO 2 DE REDES

## Versão Server

__Nomes:__

* Augusto Ferreira
* Bruno Moreira
* Gabriel Nori
* Débora Rossini ✝

__Professor:__

* Luiz Henrique

In [2]:
class DataFrame:
    __delimiter = None
    __length = None
    __sequence = None
    __destination_address = None
    __source_address = None
    __payload = None
    __crc = None

    def __init__(self, length, sequence, destination_address, source_address, payload):
        self.__delimiter = "01111110"
        self.__length = length
        self.__sequence = sequence
        self.__destination_address = destination_address
        self.__source_address = source_address
        self.__payload = payload
        self.__crc = ""

    def set__crc(self, crc):
        self.__crc = crc

    def get_crc(self):
        return self.__crc

    def get_length(self):
        return self.__length

    def get_sequence(self):
        return self.__sequence

    def get_source_address(self):
        return self.__source_address

    def get_destination_address(self):
        return self.__destination_address

    def last_bit_sequence(self):
        return self.__sequence[7]

    def first_bit_sequence(self):
        return self.__sequence[0]

    def return_data_frame(self):
        frame = ''
        frame += self.__delimiter
        frame += self.__length
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address
        frame += self.__payload
        frame += self.__crc
        return frame

    def return_data_frame_to_crc(self):
        frame = ''
        frame += self.__length
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address
        frame += self.__payload
        return frame

    @staticmethod
    def string_to_DataFrame(string):
        aux = len(string)
        delimiter = string[0:8]
        length = string[8:16]
        sequence = string[16:24]
        destination_address = string[24:56]
        source_address = string[56:88]
        payload = string[88:aux - 16]
        crc = string[aux - 16:aux]
        dataFrame = DataFrame(length, sequence, destination_address, source_address, payload)
        dataFrame.set__crc(crc)

        return dataFrame

    def get_payload(self):
        return self.__payload

    
    
class ConfirmationFrame:
    __delimiter = None
    __sequence = None
    __destination_address = None
    __source_address = None

    def __init__(self, sequence, destination_address, source_address):
        self.__delimiter = "01111110"
        self.__sequence = sequence
        self.__destination_address = destination_address
        self.__source_address = source_address

    def last_bit_sequence(self):
        return self.__sequence[7]

    def first_bit_sequence(self):
        return self.__sequence[0]

    def return_confirmation_frame(self):
        frame = ''
        frame += self.__delimiter
        frame += self.__sequence
        frame += self.__destination_address
        frame += self.__source_address
        return frame

    @staticmethod
    def string_to_ConfirmationFrame(string):
        delimiter = string[0:8]
        sequence = string[8:16]
        destination_address = string[16:48]
        source_address = string[48:80]
        confirmationFrame = ConfirmationFrame(sequence, destination_address, source_address)

        return confirmationFrame

    
    
import socket
import time


class MySocketTCP:
    __port = 18181

    def __init__(self, host='127.0.0.1', port=__port):
        self.__host = host
        self.__port = port

    @staticmethod
    def initsocket():
        time.sleep(3)
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        return s

    def send(self, msg):
        s = self.initsocket()

        s.connect((self.__host, self.__port))
        s.sendall(msg.encode())
        # data = s.recv(1024)
        # s.close()

    def receive(self):
        s = self.initsocket()
        s.bind((self.__host, self.__port))
        s.listen(1)
        data = ''


        # recebe mensagem
        while True:
            # recebe conexao
            conn, addr = s.accept()
            data = conn.recv(3000)
            if not data:
                continue
            conn.close()
            x = data.decode('ascii')
            s.close()
            return x

        
        
class CRC:
    """description of class"""

    @staticmethod
    def crc(msg):

        # Append the code to the message. If no code is given, default to '000'

        div = '11000000000000101'
        code = (len(div) - 1) * '0'
        msg = msg + code

        msg = list(msg)
        div = list(div)

        # Loop over every message bit (minus the appended code)
        for i in range(len(msg) - len(code)):
            # If that messsage bit is 1, perform modulo 2 multiplication
            if msg[i] == '1':
                for j in range(len(div)):
                    # Perform modulo 2 multiplication on each index of the divisor
                    msg[i + j] = str((int(msg[i + j]) + int(div[j])) % 2)

        # Output the last error-checking code portion of the message generated
        return ''.join(msg[-len(code):])

    
    
class Conversion:

    @staticmethod
    def decimal_to_binary(decimal):
        temp = ""
        while decimal > 0:
            s = str(int(decimal % 2))
            temp += s
            decimal = int(decimal / 2)
        if len(temp) < 8:
            s = ""
            i = 0
            size = len(temp)
            loop = 8 - size
            while i < loop:
                s += "0"
                i += 1
            temp = temp + s
        binary = ""
        i = 0
        while i < 8:
            binary += str(temp[7 - i])
            i += 1
        return binary

    @staticmethod
    def decimal_to_hexadecimal(decimal):
        pass

    @staticmethod
    def binary_to_decimal(binary):
        decimal = 0
        for bit in binary:
            if bit == "1":
                decimal = decimal * 2 + 1
            else:
                decimal = decimal * 2
        return decimal

    @staticmethod
    def binary_to_hexadecimal(binary):
        return hex(int(binary, 2))

    @staticmethod
    def hexadecimal_to_string(hexadecimal):
        return bytearray.fromhex(hexadecimal).decode('ascii')

    @staticmethod
    def binary_to_string(bin):
        hex = Conversion.binary_to_hexadecimal(bin)
        s = Conversion.hexadecimal_to_string(hex[2:])
        return s

    @staticmethod
    def hexadecimal_to_binary(hexadecimal):
        a = bin(int(hexadecimal, 16))
        return a.replace('b', '')

    @staticmethod
    def string_to_hexadecimal(string):
        return string.encode('ascii').hex()

    @staticmethod
    def string_to_binary(string):
        string = Conversion.string_to_hexadecimal(string)
        return Conversion.hexadecimal_to_binary(string)

    @staticmethod
    def ip_to_binary(ip):
        ip = ip.split('.')
        binary = ""
        for num in ip:
            binary += Conversion.decimal_to_binary(int(num))
        return binary

    @staticmethod
    def binary_to_ip(bin):
        a = bin[0:8]
        b = bin[8:16]
        c = bin[16:24]
        d = bin[24:32]

        A = str(Conversion.binary_to_decimal(a))
        B = str(Conversion.binary_to_decimal(b))
        C = str(Conversion.binary_to_decimal(c))
        D = str(Conversion.binary_to_decimal(d))

        ip = A + '.' + B + '.' + C + '.' + D
        return ip

    
    
import math
import random


class Fisica:
    __TAXA = 0.1

    @staticmethod
    def add_noise(msg):
        size = len(msg)
        quantidade = math.ceil(Fisica.__TAXA * size)

        for i in range(quantidade):
            pos = random.randrange(0, len(msg))
            l = list(msg)
            l[pos] = Fisica.invert(l[pos])
            msg = ''.join(l)

        return msg

    @staticmethod
    def invert(char):
        if char == '1':
            return '0'
        else:
            return '1'


from ConfirmationFrame import ConfirmationFrame
from DataFrame import DataFrame
from Conversion import Conversion
from MySocketTCP import MySocketTCP
from CRC import CRC
from Fisica import Fisica


class Enlace:

    def __init__(self, address):
        __destination_address = address[0]
        __source_address = address[1]

    @staticmethod  ## Servidor que recebe o frame e responde com o ACK
    def data_request():
        mensagem = ''

        while True:

            socket_receber = MySocketTCP('127.0.0.1')
            resposta = socket_receber.receive()

            if resposta == '#':
                print('CLIENTE DESCONECTOU')
                break

            r2 = DataFrame.string_to_DataFrame(resposta)

            r2_payload_bin = r2.get_payload()
            r2_crc_recebido = r2.get_crc()
            crc_novo = CRC.crc(r2.return_data_frame_to_crc())

            source_bin = r2.get_destination_address()

            destination_bin = r2.get_source_address()
            destination = Conversion.binary_to_ip(destination_bin)

            n_ack = 0
            sequence = r2.first_bit_sequence()
            if crc_novo != r2_crc_recebido:
                sequence += '0000000'
                n_ack = 0
                # manda quadro de confirmação com ack = 0
            else:
                sequence += '0000001'
                n_ack = 1

                mensagem += Conversion.binary_to_string(r2_payload_bin)
                print('MENSAGEM RECEBIDA:', Conversion.binary_to_string(r2_payload_bin))
                print('FRAME OK')

            ack_frame = ConfirmationFrame(sequence, destination_bin, source_bin)



            print('ENVIANDO CONFIRMATION FRAME:   ', ack_frame.return_confirmation_frame())

            ack_socket = MySocketTCP(destination)
            ack_socket.send(ack_frame.return_confirmation_frame())

            print('CONFIRMATION FRAME ENVIADO COM ACK = ', n_ack)

        return(mensagem)

    @staticmethod  ## Cliente que manda
    def data_indication(destination_address, source_address, l_sdu):
        for segment in l_sdu:

            print('MENSAGEM A SER ENVIADA:   ', segment)

            # Delimitador já é inserido no quadro

            # comprimento do payload
            length = Conversion.decimal_to_binary(len(segment))

            # campo de sequencia TODO
            #          SEQ----ACK
            sequence = '00000000'

            # destino do quadro
            destination_address_frame = Conversion.ip_to_binary(destination_address)

            # origem do quadro
            source_address_frame = Conversion.ip_to_binary(source_address)

            # dados convertidos
            payload = Conversion.string_to_binary(segment)

            # cria o frame
            frame = DataFrame(
                length,
                sequence,
                destination_address_frame,
                source_address_frame,
                payload)

            # Calcula o crc
            MyCRC = CRC.crc(frame.return_data_frame_to_crc())
            frame.set__crc(MyCRC)

            # cria a string para enviar
            frame_str = frame.return_data_frame()

            socket_envio = MySocketTCP(destination_address)

            if segment == 'FIM':
                socket_envio.send('#')
            else:
                # Adicionar ruido ao quadro
                # corrompe = random.randrange(0, 2)
                # if corrompe == 1:
                #     print('---houve ruido----')
                #     frame_str = Fisica.add_noise(frame_str)

                # ENVIA!!!
                print('FRAME:    ', frame_str)
                socket_envio.send(frame_str)

                print('FRAME ENVIADO')

                receive_ack_socket = MySocketTCP()
                got_it = receive_ack_socket.receive()

                OK = ConfirmationFrame.string_to_ConfirmationFrame(got_it)

                ACK = OK.last_bit_sequence()

                print('ACK RECEBIDO: ', ACK)

                
from Enlace import Enlace
import math


class User:
    __MAX_SIZE_BITS = 63

    # __MAX_SIZE_BITS = 255

    def __init__(self, host):
        self.__host = host

    def send_message(self, message, destination_address):
        n_segments = int(math.ceil(len(message) / self.__MAX_SIZE_BITS))
        fragments = []
        for i in range(n_segments):
            fragments.append(message[i * self.__MAX_SIZE_BITS:(i + 1) * self.__MAX_SIZE_BITS])

        fragments.append('FIM')
        Enlace.data_indication(destination_address, self.__host, fragments)

    def receive_message(self):
        message = Enlace.data_request()

        print('\n\nMENSAGEM RECEBIDA: ', message)

        
        
        

In [3]:
from User import User

import socket

HOST_SOURCE = '127.0.0.1'


def main():
    server = User(HOST_SOURCE)

    server.receive_message()


if __name__ == '__main__':
    main()


MENSAGEM RECEBIDA: O objetivo deste trabalho e implementar uma camada de enlace co
FRAME OK
ENVIANDO CONFIRMATION FRAME:    01111110000000010111111100000000000000000000000101111111000000000000000000000001
CONFIRMATION FRAME ENVIADO COM ACK =  1
MENSAGEM RECEBIDA: m servico de comunicacao confiavel por meio de sockets. O proto
FRAME OK
ENVIANDO CONFIRMATION FRAME:    01111110000000010111111100000000000000000000000101111111000000000000000000000001
CONFIRMATION FRAME ENVIADO COM ACK =  1
MENSAGEM RECEBIDA: colo emprega transmissao confiavel do tipo Stop-and-Wait. Alem 
FRAME OK
ENVIANDO CONFIRMATION FRAME:    01111110000000010111111100000000000000000000000101111111000000000000000000000001
CONFIRMATION FRAME ENVIADO COM ACK =  1
MENSAGEM RECEBIDA: da experiencia no dsenvolvimento de aplicacoes via socket, este
FRAME OK
ENVIANDO CONFIRMATION FRAME:    01111110000000010111111100000000000000000000000101111111000000000000000000000001
CONFIRMATION FRAME ENVIADO COM ACK =  1
MENSAGEM RECEBIDA:  